In [1]:
import torch 
from models.longformer.longformer_encoder_decoder import LongformerEncoderDecoderModel
from vocabs.vocab import Vocab
from torch import nn

class SummarizationWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def forward(self, input_ids):
        return self.model.predict(input_ids)

ImportError: cannot import name 'Vocab' from partially initialized module 'vocabs.vocab' (most likely due to a circular import) (/home/trang/project/ViPhoLM/vocabs/vocab.py)

In [ ]:
# Load vocab
vocab = Vocab.load("checkpoints/longfoLongformerEncoderDecoder_Wikilingualrmer/vocab.bin")  # hoặc path bạn lưu vocab

# Load config
config_dict = torch.load("checkpoints/LongformerEncoderDecoder_Wikilingual/best_model.pth", map_location="cpu")
config = LongformerEncoderDecoderConfig(config_dict["config"])  # đảm bảo truyền đúng config

# Load model
model = LongformerEncoderDecoderModel(config, vocab)
model.load_state_dict(config_dict["model_state_dict"])
model.eval()


In [ ]:
# Dummy input tensor (batch_size=1, seq_len=512)
dummy_input = torch.randint(low=0, high=vocab.vocab_size, size=(1, 512))

# Wrap model
wrapped_model = SummarizationWrapper(model)

# Trace
scripted_model = torch.jit.trace(wrapped_model, dummy_input)

# Lưu TorchScript
torch.jit.save(scripted_model, "checkpoints/LongformerEncoderDecoder_Wikilingual/longformer_summarizer.pt")
print("TorchScript model saved!")
